# Binary Black Hole Evolution: Stable Mass Transfer vs Common Envelope
**Group 3 authors:** 
- Bonasera Elias Maria (2149751)
- Casellato Alberto (2139206)
- Garbin Nicola (2156363)
- Tamanna Tasneem (2040474)


## 1. Goal of the Project

The goal of our project is, given a set of simulated binary black hole (BBH) systems, to understand how the physical properties of the systems are correlated with their evolutionary path (Stable Mass Transfer or Common Envelope). 

Our task is divided into two parts:

- Classify the BBHs based on their evolutionary track 
- Analyze how different physical parameters affect the evolution of BBH mergers

To perform this, we will use visualization techniques and machine learning methods (e.g., Random Forest) to determine the most significant features that influence BBH mergers evolution.

## 2. Theoretical Background

A **binary star system** is composed of two stars that orbit around the center of mass of the system. The most massive star is called the **primary star** (or *donor star*), and the other one is the **secondary star** (or *accretor*). There are different ways in which a star can lose mass; the most common are:

- **Stellar winds**: which occours because of single star evolution. This mass loss follows the relation $\dot{M} \propto Z^p$, with $p$ a coefficient that depends on the star's temperature.
- **Roche Lobe overflow**: which happens when a star increases in size due to natural evolution, exceeding a certain boundary. This boundary is defined by two teardrop-shaped equipotential surfaces that connect the two stars, within which the stars are confined. Each teardrop is called the **Roche Lobe** of a star, and the point where they meet is the **Lagrangian point L1**. When stellar matter reaches L1, it is no longer gravitationally bound to the donor star and is transferred to the accretor, starting a process of mass transfer known as **Roche Lobe overflow**.

### 2.1 Roche Lobe Overflow
The donor star has the biggest Roche lobe, but it is the first one to fill it because the more a star is massive and the faster is its evolution.\
From the literature, we get that $R_L \propto a \ f(q)$, where:
- $R_L$ is the Roche Lobe radius,
- $a$ is the semi-major axis of the system,
- $q$ is the mass ratio of the two stars.

<img src="images/roche_potential.png" width="400">

There are two types of responses a star can have to mass loss, depending on its internal structure:

- **Stable Mass Transfer (SMT)**: The star responds to mass loss by adjusting its size, leading to a gradual transfer of mass to the secondary star. This results in minimal mass loss from the system.
- **Common Envelope (CE)**: The primary star expands uncontrollably, leading to an unstable phase where the outer layers engulf both stars, forming a common envelope.

### 2.2 Common Envelope
The common envelope that surrounds the two stars extends beyond the system's orbit. Due to angular momentum conservation, it rotates slower than the two stars inside. This difference in velocity creates friction forces, which extract energy from the system, causing the two stars to spiral inward.

There are two possible outcomes:
- The system transfers enough energy to the common envelope to eject it outward, leaving behind a shrunken orbit and a significant mass loss.
- The system fails to transfer enough energy, leading to the merging of both stars inside the envelope, forming a single object surrounded by the ejected material.

The **$\alpha$ parameter** is a free parameter that quantifies the efficiency of energy transfer between the system and the common envelope. It determines how much orbital energy is used to eject the envelope: the larger it is, the lesser orbital energy is required.

<img src="images/common_envelope_evolution.png" width="400">


<!--
Da aggiungere alle conclusioni:
- CE processo più dissipativo, infatti crea buchi neri di massa più piccola, mentre SMT crea BH più massicci
-->


## 3. Datasets

Data is contained in the folder `data`, which contains four folders, one for each value of $\alpha$ (0.5, 1, 3, 5): *A05*, *A1*, *A3* and *A5*. Inside every folder, there are 12 files named MTCE_BBHs_*.txt, where \* is a number that represents the metallicity, from 0.0002 to 0.02. Each file follows this structure:

### Header
- `Row 0`: Header for Row 1 
- `Row 1`: Two columns:  
  - `Column 0`: Total mass of the simulated systems ($M_{\odot}$) 
  - `Column 1`: Number of binary black hole mergers
- `Row 2`: Header for the data columns
  
### Main dataset
- **Systems ID:**
    - `Column 0`: Identifier of the binary system

- **Individual Star Properties:**
    - `Column 1`: Initial mass (ZAMS: Zero Age Main Sequence) of the primary star ($M_{\odot}$)
    - `Column 2`: Initial mass (ZAMS) of the secondary star ($M_{\odot}$)
    - `Column 3`: Mass of the black hole formed from the primary star ($M_{\odot}$)
    - `Column 4`: Mass of the black hole formed from the secondary star ($M_{\odot}$)

- **Binary System Properties:**
    - `Column 5`: Total Mass of the BBH system ($M_{\odot}$)
    - `Column 6`: Delay time (time from the formation of the binary system to the merger of the two BHs, in Myr)
    - `Column 7`: Semi-major axis of the binary at the formation of the second black hole ($R_{\odot}$)
    - `Column 8`: Orbital eccentricity of the system at the formation of the second black hole

- **Supernova Properties:**
    - `Column 9`: Magnitude of the supernova kick (km/s) for the primary black hole
    - `Column 10`: Magnitude of the supernova kick (km/s) for the secondary black hole
    - `Column 11`: Cosine of the tilt angle (before and after supernova) for the primary black hole
    - `Column 12`: Cosine of the tilt angle (before and after supernova) for the secondary black hole
    - `Columns 13-18`: x, y, z components of the center-of-mass velocity after the supernova explosion of the primary and secondary components
    - `Column 19`: Time at which the primary component undergoes a supernova
    - `Column 20`: Time at which the secondary component undergoes a supernova

- **Binary Classification Label:**
    - `Column 21`: Binary evolution path
        - `True`: The system evolved via common envelope
        - `False`: The system evolved via stable mass transfer

In [1]:
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import numpy as np
import pandas as pd
import os

## 4. Data Preprocessing and Data Visualization

### 4.1 Importing Data
We defined the function *load_data* to load the datasets as a nested dictionary, where the first keys correspond to $\alpha$ values, and the second ones are the different metallicities, with pandas dataframes as values.

Given as input a folder containing data, it first searches through every subdirectory of that folder, each representing a different $\alpha$ value, and then it reads the data files contained inside, corresponding to a different metallicity.

In this way, we can access data for a specific $\alpha$ and $Z$ as follows `data[alpha][metallicity]`.

In [2]:
def load_data(folder):
    dict_dataset = {}
    for filename1 in os.listdir(folder):
        folder_path = os.path.join(folder, filename1)
        list_dataset = {}

        for filename2 in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename2)
            df = pd.read_csv(file_path, delimiter=" ", skiprows=2)
            list_dataset[filename2.split("_")[2].split(".txt")[0]] = df
        
        dict_dataset[filename1] = list_dataset
    
    return dict_dataset

In [3]:
# plot settings
major_ticks_length = 10
minor_ticks_length = major_ticks_length / 2
label_size = 20
mpl.rcParams['axes.linewidth'] = 1.1

# loading data
data = load_data("data")

### 4.2 Visualizing Data

#### How the `plot_histogram` function works

- The function takes as input a list of two axes, two datasets, a list of metallicities, the number of bins, a limit for the x-axis, a label and a title. 
- It assigns a color to each metallicity with the plasma colormap. 
- If a limit for the x-axis is given, it determines also the histogram bins. Otherwise, the function uses the predefined binning. 
- The function plots on the first axis histograms for every metallicity of SMT systems and on the second axis  it does the same for CE systems.
- On the y-axis a logarythmic scale is applied, since the distributions cover a wide range of orders of magnitude.


#### How the `plot_stackplot` function works

- The function takes as input a dictionary containing the fractions of systems for different $\alpha$ values, a list of metallicities, and a color palette.
- The function generates a figure with four stackplots. Each plot corresponds to a different value of the $\alpha$ parameter and shows the fraction of SMT systems and CE systems, indicated with different colors, as a function of metallicity.
- It applies a logarithmic scale on the x-axis, where metallicities are represented.

In [4]:
def plot_histogram(axes, data_1, data_2, metallicities, bins=100, xlim=None, xlabel="", title=""):
    """
    Plots histograms for two sets of data with the same bins and xlim.
    """ 
    cmap = cm.plasma
    colors = [cmap(i / len(data_1)) for i in range(len(data_1))]
    if xlim:
        bins = np.linspace(xlim[0], xlim[1], bins)
    
    # plots
    for i in range(len(data_1)):
        for k,data in enumerate([data_1, data_2]):
            axes[k].hist(data[i], bins=bins, histtype="step", label=f"Z={metallicities[i]}", 
                         edgecolor=colors[i], linewidth=2)
    
    # axes settings
    axes[0].tick_params(axis='both', which='both', direction="in", top=True, right=True, labelbottom=False)
    axes[1].tick_params(axis='both', which='both', direction="in", top=True, right=True, labelbottom=True)
    
    axes[1].set_xlabel(xlabel, fontsize=label_size)

    for ax in axes:
        if xlim:
            ax.set_xlim(xlim)
        ax.set_yscale("log")
        ax.tick_params(which="major", length=major_ticks_length, labelsize=label_size)
        ax.tick_params(which="minor", length=minor_ticks_length)

    plt.tight_layout()


def plot_stackplot(fractions_dict, metallicities, colors):
    """
    Plots a stackplot showing fractions of stable mass transfer and common envelope systems.
    """    
    fig, axes = plt.subplots(1, 4, figsize=(6*4, 8), sharex=True, sharey=True, 
                             gridspec_kw={"hspace": 0, "wspace": 0})
    
    sorted_alpha = sorted(list(fractions_dict.keys()), key=lambda x: float(x[1:]))
    
    for i,alpha in enumerate(sorted_alpha):
        axes[i].stackplot(
            metallicities,
            fractions_dict[alpha][0],
            fractions_dict[alpha][1],
            labels=["Stable Mass Transfer", "Common Envelope"],
            colors=colors,
            edgecolor="k",
            alpha=0.8
        )    
        axes[i].tick_params(which="major", direction="in", length=major_ticks_length, labelsize=label_size+2)
        axes[i].tick_params(which="minor", direction="in", length=minor_ticks_length)
        
        if i == 0:
            axes[i].legend(loc="upper right", fontsize=label_size)
            axes[i].set_ylabel("Fraction", fontsize=label_size+7, labelpad=20)

        axes[i].set_xlim(min(metallicities), max(metallicities))
        axes[i].set_ylim(0,1.)
        axes[i].set_xscale("log")
        axes[i].set_title(f"$\\alpha$ = {alpha[1:]}", fontsize=label_size+3)
        axes[i].grid(linestyle="--", which="both", linewidth=1, color="dimgrey")
        
    fig.supxlabel("Metallicity", fontsize=label_size+7, y=0.01)
    fig.suptitle("Fraction of Systems with SMT and CE", fontsize=label_size+10, y=1)
    
    plt.tight_layout()
    plt.savefig("images/stackplot.png", bbox_inches='tight')
    plt.close(fig)

#### Property Distributions  

To analyze the influence of the different physical properties on the evolution of BBH systems, we will visualize the distribution of specific properties for various values of $\alpha$, considering different metallicities and evolutionary pathways (SMT or CE). We will show different figures, each consisting of two rows and four columns. The first row represents systems that evolve through SMT and the second row corresponds to systems undergoing CE evolution. Each column corresponds to a different value of the $\alpha$ parameter. 

The histograms inside each plot show the distribution of a specific property for different values of metallicities. Lower metallicities are represented by darker colors, while higher metallicities appear in lighter shades. High metallicities (Z) are only observed in CE systems, with a decreasing trend as $\alpha$ increases.

Since the $\alpha$ parameter only affects CE evolution, the first row (SMT) is expected to remain consistent across different $\alpha$ values. This consistency allows for a direct comparison with the second row (CE), where variations due to $\alpha$ can be observed. 

In [7]:
properties_alpha = {}
fractions_alpha = {}
list_metallicites = []

alpha_sorted = sorted(data.keys(), key=lambda x: float(x[1:]))
for a in alpha_sorted:
    dict_df = data[a]
    alpha = float(a[1:])
    
    masses_smt = []
    masses_ce = []
    masses2_smt = []
    masses2_ce = []
    masses_sum_smt=[]
    masses_sum_ce=[]
    masses_rate_smt = []
    masses_rate_ce = []
    masses_BH_smt = []
    masses_BH_ce = []
    masses_rate_BH_smt = []
    masses_rate_BH_ce = []
    delay_time_smt = []
    delay_time_ce = []
    list_sma_smt =[]
    list_sma_ce = []
    list_e_smt =[]
    list_e_ce = []
    
    smt_fractions = []
    ce_fractions = []
    
    metallicities = sorted(dict_df.keys(), key=lambda x: float(x))
    list_metallicites = [float(m) for m in metallicities]
    for i,z in enumerate(metallicities):
        df = dict_df[z]
        
        # data selection
        df_smt = df[df["col.21:CE"]==0]
        df_ce = df[df["col.21:CE"]==1]
        smt_count = df_smt.shape[0]
        ce_count = df_ce.shape[0]
        total_count = smt_count + ce_count
        smt_fractions.append(smt_count/total_count)
        ce_fractions.append(ce_count/total_count)
        
        m1_smt = df_smt["col.1:m1ZAMS/Msun"]
        m1_ce = df_ce["col.1:m1ZAMS/Msun"]
        m2_smt = df_smt['col.2:m2ZAMS/Msun']
        m2_ce = df_ce['col.2:m2ZAMS/Msun']
        
        m1_BH_smt = df_smt["col.3:m1rem/Msun"]
        m1_BH_ce = df_ce["col.3:m1rem/Msun"]
        m2_BH_smt = df_smt['col.4:m2rem/Msun']
        m2_BH_ce = df_ce['col.4:m2rem/Msun']
        
        delay_smt = df_smt['col.6:delay_time/Myr']/1000
        delay_ce = df_ce['col.6:delay_time/Myr']/1000
        
        sma_smt = df_smt['col.7:sma/Rsun']
        sma_ce = df_ce['col.7:sma/Rsun']
        
        e_smt = df_smt['col.8:ecc']
        e_ce = df_ce['col.8:ecc']
        
        # mass data
        masses_smt.append(m1_smt)
        masses_ce.append(m1_ce)
        
        # mass 2 data
        masses2_smt.append(m2_smt)
        masses2_ce.append(m2_ce)
        
        # mass sum
        masses_sum_smt.append(m1_smt + m2_smt)
        masses_sum_ce.append(m1_ce + m2_ce)
        
        # mass rate data
        masses_rate_smt.append(m2_smt/m1_smt)
        masses_rate_ce.append(m2_ce/m1_ce)
        
        # mass BH data
        masses_BH_smt.append(m1_BH_smt)
        masses_BH_ce.append(m1_BH_ce)
        
        # mass rate BH data
        masses_rate_BH_smt.append(m2_BH_smt/m1_BH_smt)
        masses_rate_BH_ce.append(m2_BH_ce/m1_BH_ce)
        
        # delay time data
        delay_time_smt.append(delay_smt)
        delay_time_ce.append(delay_ce)
        
        # semi-major axis data
        list_sma_smt.append(sma_smt)
        list_sma_ce.append(sma_ce)
        
        # eccentricity data
        list_e_smt.append(e_smt)
        list_e_ce.append(e_ce)
    
    properties_alpha[a] = [masses_smt, masses_ce, masses2_smt, masses2_ce, masses_sum_smt, masses_sum_ce, 
                           masses_rate_smt, masses_rate_ce, masses_BH_smt, masses_BH_ce, 
                           masses_rate_BH_smt , masses_rate_BH_ce , list_e_smt, list_e_ce]
    fractions_alpha[a] = [smt_fractions, ce_fractions]
    
xlabel_list = [
    "Primary Star Mass [M$_\\odot$]", 
    "Secondary Star Mass [M$_\\odot$]",
    "Star Mass Sum [M$_\\odot$]",
    "Star Mass Ratio [$M_2/M_1$]", 
    "Primary BH Mass [M$_\\odot$]", 
    "BH Mass Ratio [$M_2/M_1$]",
    "Eccentricity"]

titles_list = [
    "ZAMS Mass Distribution of Primary Stars",
    "ZAMS Mass Distribution of Secondary Stars",
    "ZAMS Mass Sum",
    "ZAMS Mass Ratio of Stars",
    "Mass Distribution of Primary BHs",
    "Mass Ratio of BHs",
    "Orbital Eccentricity of BBHs"
]

bins_list=[50, 50, 50, 50, 50, 50, 30]
xlimlist=[[10, 160], [-2, 162], [10, 310], [0.1, 1.05], [0,49.5], [0, 4.9], None]
maxlen=50

for k in range(0, len(properties_alpha["A1"]) - 1, 2):        
    fig, axes = plt.subplots(2, 4, figsize=(28, 12), sharex=True, sharey=True, 
                             gridspec_kw={"hspace": 0, "wspace": 0})

    for i, alpha in enumerate(list(properties_alpha.keys())):
        pr1, pr2 = properties_alpha[alpha][k], properties_alpha[alpha][k+1]
        for j in range(len(pr1)):
            if len(pr1[j])<maxlen:
                pr1[j]=[]
            if len(pr2[j])<maxlen:
                pr2[j]=[]
        plot_histogram([axes[0][i], axes[1][i]], pr1, pr2, metallicities, bins=bins_list[k//2], xlim=xlimlist[k//2])
        
        if i == 0:
            axes[0][i].set_ylabel("$N$ Systems\n (Stable Mass Transfer)", fontsize=label_size+7)
            axes[1][i].set_ylabel("$N$ Systems\n (Common Envelope)", fontsize=label_size+7)
        
        axes[0][i].set_title(f"$\\alpha$ = {alpha[1:]}", fontsize=label_size+3)

    fig.supxlabel(xlabel_list[k//2], fontsize=label_size+10, y=-0.04)  
    fig.suptitle(titles_list[k//2], fontsize=label_size+15, y=1.05)

   
    handles, labels = axes[0][-1].get_legend_handles_labels()
    fig.legend(handles, labels, loc="center right", fontsize=label_size, title="Metallicity", 
        title_fontsize=18, frameon=False, bbox_to_anchor=(0.98, 0.5))

    plt.subplots_adjust(right=0.88)

    fname = xlabel_list[k//2]
    plt.savefig(f"images/{fname.split(' [')[0]}.png", bbox_inches='tight')
    plt.close(fig)

plot_stackplot(fractions_alpha, list_metallicites, colors=["#298c8c", "#f1a226"])

<img src="images/Primary Mass.png">

The figure illustrates the distribution of the ZAMS masses of the primary star. As $\alpha$ increases, there is a reduction in the number of primary stars with higher mass formed through CE. When $\alpha \gtrsim 3$, systems with a high primary star mass that evolve through SMT become more prevalent than those that undergo CE.

---

<img src="images/Secondary Mass.png">

The figure shows the distribution of the ZAMS masses of the secondary star. For systems with the mass of the secondary star $M_2 \gtrsim 100\ M_{\odot}$ no SMT occurs.
As $\alpha$ increases, the CE distribution tends to follow the SMT distribution, indicating that an efficient envelope ejection produces systems that resemble SMT binaries.

---

<img src="images/Mass Ratio.png">

The figure shows the distribution of the ZAMS mass ratio, representing the secondary star mass over the primary star mass. For stars with similar masses ($q \ge 0.6$), no stable mass transfer occurs. The lower the metallicity of the binary system, the greater the possible mass difference between the two stars.

---

<img src="images/Primary BH Mass.png">

The figure shows the distribution of the masses of the primary black holes.\
Primary stars with high metallicity form black holes with lower masses, as a result of stronger stellar winds. The primary BH masses of SMT systems follow a uniform distribution of black hole masses, with no masses below $10\ M_{\odot}$.\
By comparing the mass range of the primary black holes of this figure with the one of their progenitor stars (first figure), we can observe the significant mass loss happening during the evolution of the binary systems.

---

<img src="images/BH Mass Ratio.png">

The figure shows the distribution of the black hole mass ratio, defined as the secondary black hole mass over the primary black hole mass. The mass ratio tends to invert once a binary black hole system is formed, with the ratio sometimes reaching values greater than 1. In low-metallicity CE systems, the ratio remains between 0 and 1. For SMT systems, the mass ratio stays small, which implies a minimal change in the mass ratio of the system, as opposed to CE systems, which experience significant mass loss, leading to an inversion of the ratio ($M_2 > M_1$).
<!--The reader can appreciate the rare but rather useful Condom Distribution.-->

---

<img src="images/Eccentricity.png">

This image shows the distribution of the orbital eccentricity of binary black holes. Systems with CE evolution occupy the entire eccentricity range, while other systems are limited to a narrower range ($e \lesssim 0.3$). Some systems have $e=1$, which indicates that they are no longer bound. 

---

#### Stackplot of Evolution vs Metallicity
Another method we use to compare the two classes is the visualization of the fraction of systems that evolved through SMT and CE as a function of metallicity. There will be a plot for each value of $\alpha$ and the fraction of systems is normalized. In yellow are represented the systems that underwent CE and in green the ones that formed through SMT. 

<img src="images/stackplot.png">

There is an overall majority of systems with CE for all metallicities and for every value of the alpha parameter. But, as $\alpha$ increases, in the region between $10^{-3}$ and $10^{-2}$, the fraction of systems with SMT increases and exceeds the fraction of CE systems.